<a href="https://colab.research.google.com/github/ArifAygun/Webscraping/blob/main/AA_Global_Companies__Ranked_by_Revenue_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/My Drive/Web_Scraping/Companiesmarketcap/

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/Web_Scraping/Companiesmarketcap


In [20]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from pprint import pprint

In [21]:
base_site = f"https://companiesmarketcap.com/largest-companies-by-revenue/"
response = requests.get(base_site)
html = response.content
soup = BeautifulSoup(html, "html.parser")

In [22]:
def pageCountFound():
    base_site1 = f"https://companiesmarketcap.com/largest-companies-by-revenue/"
    response = requests.get(base_site1)
    html = response.content
    soup = BeautifulSoup(html, "html.parser")
    rowCount = int(soup.find('span', {'class':'companies-count'}).text.replace(',', ''))
    pageCount = (rowCount + 99) // 100  # Calculate pageCount based on total rows
    return pageCount

pageCountFound()

80

In [23]:
ranks_list = soup.find_all('td', {'class':'rank-td td-right'})
ranks = [i.text.strip() for i in ranks_list]
#ranks

name_list = soup.find_all('td', {'class':'name-td'})
names = [i.select('div', {'class':'company-name'})[2].text.strip() for i in name_list]
#names

name_link_list = soup.find_all('div', {'class':'name-div'})
name_links = [i.find("a").get("href") for i in name_link_list]
#name_links

percent_links = soup.find_all('span', {'class':'percentage-red'})
percents = [i.text for i in percent_links]
#percents

country_links = soup.find_all('span', {'class':'responsive-hidden'})
countries = [i.text.strip() for i in country_links]
#countries

In [24]:
cap_links = soup.find_all('td', {'class':'td-right'})
caps_links_list = [i.text.strip() for i in cap_links]

prices = [caps_links_list[i] for i in range(2, len(caps_links_list), 3)]
revenues = [caps_links_list[i] for i in range(1, len(caps_links_list), 3)]
ranks = [caps_links_list[i] for i in range(0, len(caps_links_list), 3)]
#prices
#revenues
#ranks

In [42]:
def pageCountFound():
    base_site1 = f"https://companiesmarketcap.com/largest-companies-by-revenue/"
    response = requests.get(base_site1)
    html = response.content
    soup = BeautifulSoup(html, "html.parser")
    rowCount = int(soup.find('span', {'class':'companies-count'}).text.replace(',', ''))
    pageCount = (rowCount + 99) // 100  # Calculate pageCount based on total rows
    return pageCount

def finalPulling():
    df_out = pd.DataFrame()
    pageCount = pageCountFound()

    for pageNo in range(1, pageCount + 1):
        base_site = f"https://companiesmarketcap.com/largest-companies-by-revenue/?page={pageNo}"
        response = requests.get(base_site)
        html = response.content
        soup = BeautifulSoup(html, "html.parser")
        tbody = soup.find('tbody')

        ranks_list = tbody.find_all('td', {'class':'rank-td td-right'})
        ranks = [i.text.strip() for i in ranks_list]

        country_links = tbody.find_all('span', {'class':'responsive-hidden'})
        countries = [i.text.strip() for i in country_links]

        name_list = tbody.find_all('td', {'class':'name-td'})
        names = [i.select('div', {'class':'company-name'})[2].text.strip() for i in name_list]

        links = tbody.find_all('td', {'class':'td-right'})
        links_list = [i.text.strip() for i in links]

        prices = [links_list[i] for i in range(2, len(links_list), 3)]
        revenues = [links_list[i] for i in range(1, len(links_list), 3)]
        ranks = [links_list[i] for i in range(0, len(links_list), 3)]

        df_in = pd.DataFrame(zip(ranks, names, revenues, prices, percents, countries), columns =["ranks","names", "revenues", "prices", "percents", "countries"])
        df_out = pd.concat([df_out, df_in], axis = 0)

    df_out.to_csv("df_out.csv")
    return df_out

finalPulling()

,ranks,names,revenues,prices,percents,countries
0,1,Nornickel,$1.184 T,$211.35,1.53%,Russia
1,2,Walmart,$630.79 B,$164.47,0.47%,USA
2,3,Amazon,$538.04 B,$137.02,0.16%,USA
3,4,Saudi Aramco,$535.54 B,$9.15,0.27%,S. Arabia
4,5,Sinopec,$469.29 B,$0.84,0.82%,China
...,...,...,...,...,...,...
33,7934,Western Asset Mortgage Capital,-$21.37 M,$10.11,1.21%,USA
34,7935,Orchid Island Capital,-$27.66 M,$9.44,0.41%,USA
35,7936,Intouch Holdings,-$43.44 M,$2.06,0.39%,Thailand
36,7937,American Virtual Cloud,-$49.64 M,$0.60,2.21%,USA
